Script to create utility equations for Alogit control files. 
Run this, then copy-paste into .alo file. 

In [1]:
import numpy as np
import pandas as pd

In [2]:
vehtype_names = ['Car', 'Van', 'SUV', 'Pickup', 'Motorcycle']
vehtypes  = ['car', 'van', 'suv', 'pu', 'mc']
vehtypes2 = ['car', 'car', 'suvpu', 'suvpu', 'mc']

# summary vs aggregate fuel types
# separate alternatives, but merge coefficients as needed
fueltype_names = ['Gas', 'Diesel', 'Hybrid', 'Plug In Electric', 'Battery Electric']
fueltypes1 = ['gas', 'dsl', 'hyb', 'pev', 'bev']
fueltypes2 = ['gas', 'dsl', 'hyb', 'ev', 'ev']
fueltypes3 = ['gas', 'gas', 'hyb', 'ev', 'ev']

# ages 1 through 20+
ages = range(1,21)

In [3]:
# define arrays to use as included files
veh_attr_base = pd.read_csv('epa_vehicle_data.csv')

with open('vehtype_base.array', 'w') as file:
    file.write("$ARRAY NMAKES(501) \n")
    file.write("$ARRAY NMODELS(501) \n")
    file.write("$ARRAY MPG(501) \n")
    file.write("$ARRAY RANGE(501) \n")
    file.write("$ARRAY CHARGE240(501) \n")
    
    file.write("\n")    
    for i, row in veh_attr_base.iterrows():      
        file.write("NMAKES(" + str(i+1) + ') = ' + str(row['NumMakes']) + '\n')   
        
    file.write("\n")
    for i, row in veh_attr_base.iterrows():              
        file.write("NMODELS(" + str(i+1) + ') = ' + str(row['NumModels']) + '\n')
        
    file.write("\n")
    for i, row in veh_attr_base.iterrows():      
        file.write("MPG(" + str(i+1) + ') = ' + str(row['MPG']) + '\n')    
        
    file.write("\n")
    for i, row in veh_attr_base.iterrows():      
        file.write("RANGE(" + str(i+1) + ') = ' + str(row['Range']) + '\n')   
        
    file.write("\n")
    for i, row in veh_attr_base.iterrows():       
        file.write("CHARGE240(" + str(i+1) + ') = ' + str(row['charge240']) + '\n')  
        

In [4]:
# define arrays to use as included files
veh_attr_base = pd.read_csv('epa_vehicle_data.csv')

with open('num_models.array', 'w') as file:
    file.write("$ARRAY NMODELS(501) \n")
    
    file.write("\n")
    for i, row in veh_attr_base.iterrows():              
        file.write("NMODELS(" + str(i+1) + ') = ' + str(row['NumModels']) + '\n')
        

In [5]:
# to get around ALOGIT limitations on the number of defined data items, hardcode 
# array values in utility equations

# define dictionaries to use below
veh_attr_base = pd.read_csv('epa_vehicle_data.csv')
numMakes  = veh_attr_base.set_index('CHOICE4B')['NumMakes'].to_dict()
numModels = veh_attr_base.set_index('CHOICE4B')['NumModels'].to_dict()
mpg       = veh_attr_base.set_index('CHOICE4B')['MPG'].to_dict()
evRange   = veh_attr_base.set_index('CHOICE4B')['Range'].to_dict()
charge240 = veh_attr_base.set_index('CHOICE4B')['charge240'].to_dict()

veh_cost_data = pd.read_csv('vehicle_cost_data.csv')
newPrice  = veh_cost_data.set_index('CHOICE4B')['NewPrice'].to_dict()
depValue  = veh_cost_data.set_index('CHOICE4B')['DepreciatedValue'].to_dict()
annMaint  = veh_cost_data.set_index('CHOICE4B')['AnnMaintCost'].to_dict()
                                                          
                                                          

In [9]:
# start from run 826, but introduce geographic constants
with open('vehtype901.util', 'w') as file:
    
    i = 0
    for v in range(1,6):
        for f in range(1,6):
            for a in range(1,21):
                i += 1
                altnum = str(i)
                alt= str(1000*v + 100*f + a)
                vt = vehtypes[v-1]
                vt2 = vehtypes2[v-1]
                ft1 = fueltypes1[f-1]
                ft2 = fueltypes2[f-1]
                ft3 = fueltypes3[f-1]
                age = str(a)
                
                # add a comment saying what the alternative is
                file.write("- " + vehtype_names[v-1] + '-' + fueltype_names[f-1] + '-Age ' + age + '\n')

                # define the utility equation
                file.write("UTIL(" + altnum + ") = 0 \n") 
                
                # only write the rest of the utility equation if the alternative is available
                if numModels[i]>0: 

                    # 1-dimensional ASCs for vehicle type, fuel type and age
                    if v>1: 
                        file.write("          + " + vt + "\n")
                    if f>1: 
                        file.write("          + " + ft1 + "\n")
                    if a>1: 
                        file.write("          + age" + age + "\n")

                    # car is the reference alternative
                    if vt!='car':
                        file.write("          + vhgtdr_" + vt + " * VHGTDR \n")
                        file.write("          + nchld_" + vt + "  * NUM_CHLD3 \n")
                        file.write("          + den3_" + vt + "   * DEN3 \n")
                        file.write("          + den4_" + vt + "   * DEN4 \n")
                        file.write("          + den5_" + vt + "   * DEN5 \n")
                        file.write("          + den6_" + vt + "   * DEN6 \n")
                        file.write("          + den7_" + vt + "   * DEN7 \n")
                        file.write("          + den8_" + vt + "   * DEN8 \n")
                        
                        file.write("          + sfo_"  + vt + "   * SFO \n")
                        file.write("          + san_"  + vt + "   * SAN \n")
                        file.write("          + atl_"  + vt + "   * ATL \n")
                        file.write("          + sea_"  + vt + "   * SEA \n")
                        file.write("          + det_"  + vt + "   * DET \n")
                        file.write("          + msp_"  + vt + "   * MSP \n")
                        file.write("          + dca_"  + vt + "   * DCA \n")
                        file.write("          + oreg_" + vt + "   * OREG \n")
                        file.write("          + ohio_" + vt + "   * OHIO \n")
                        file.write("          + smsa_" + vt + "   * SMSA \n")
                        file.write("          + rural_" + vt + "  * RURAL \n")

                    # gas (and diesel) is the reference alternative
                    if ft3!='gas':
                        file.write("          + vhgtdr_" + ft3 + " * VHGTDR \n")
                        file.write("          + nchld_" + ft3 + "  * NUM_CHLD3 \n")
                        file.write("          + den3_" + ft3 + "   * DEN3 \n")
                        file.write("          + den4_" + ft3 + "   * DEN4 \n")
                        file.write("          + den5_" + ft3 + "   * DEN5 \n")
                        file.write("          + den6_" + ft3 + "   * DEN6 \n")
                        file.write("          + den7_" + ft3 + "   * DEN7 \n")
                        file.write("          + den8_" + ft3 + "   * DEN8 \n")
                        
                        file.write("          + sfo_"  + ft3 + "   * SFO \n")
                        file.write("          + san_"  + ft3 + "   * SAN \n")
                        file.write("          + atl_"  + ft3 + "   * ATL \n")
                        file.write("          + sea_"  + ft3 + "   * SEA \n")
                        file.write("          + det_"  + ft3 + "   * DET \n")
                        file.write("          + msp_"  + ft3 + "   * MSP \n")
                        file.write("          + dca_"  + ft3 + "   * DCA \n")
                        file.write("          + oreg_" + ft3 + "   * OREG \n")
                        file.write("          + ohio_" + ft3 + "   * OHIO \n")
                        file.write("          + smsa_" + ft3 + "   * SMSA \n")
                        file.write("          + rural_" + ft3 + "  * RURAL \n")

                    # all ages are included
                    file.write("          + i025_age   * INC025a(" + age + ")\n")
                    file.write("          + i2550_age  * INC2550a(" + age + ")\n")
                    file.write("          + i100_age   * INC100150a(" + age + ")\n")
                    file.write("          + i150p_age  * INC150Pa(" + age + ")\n")
                    file.write("          + imiss_age  * INC_MISSa(" + age + ")\n")   
                    file.write("          + oneveh_age * ONEVEHa(" + age + ")\n")
                    file.write("          + vhgtdr_age * VHGTDRa(" + age + ")\n")
                    file.write("          + nchld_age  * NUM_CHLD3a(" + age + ")\n")
                    file.write("          + dstwkt_age * DSTWKTa(" + age + ")\n")
                    
                    file.write("          + sfo_age    * SFOa(" + age + ")\n")
                    file.write("          + san_age    * SANa(" + age + ")\n")
                    file.write("          + atl_age    * ATLa(" + age + ")\n")
                    file.write("          + sea_age    * SEAa(" + age + ")\n")
                    file.write("          + det_age    * DETa(" + age + ")\n")
                    file.write("          + msp_age    * MSPa(" + age + ")\n")
                    file.write("          + dca_age    * DCAa(" + age + ")\n")
                    file.write("          + oreg_age   * OREGa(" + age + ")\n")
                    file.write("          + ohio_age   * OHIOa(" + age + ")\n")
                    file.write("          + smsa_age   * SMSAa(" + age + ")\n")
                    file.write("          + rural_age  * RURALa(" + age + ")\n")    

                    # household interactions by vehicle type                
                    if vt=='van':
                        file.write("          + van_van   * OWN_VAN\n")
                        file.write("          + van_suv   * OWN_SUV\n")
                        file.write("          + van_pu    * OWN_PU\n")
                        file.write("          + van_mc    * OWN_MC\n")  
                        file.write("          + only_van  * ALL_VAN\n")                         
                    if vt=='suv':
                        file.write("          + van_suv   * OWN_VAN\n")
                        file.write("          + suv_suv   * OWN_SUV\n")
                        file.write("          + suv_pu    * OWN_PU\n")
                        file.write("          + suv_mc    * OWN_MC\n")      
                        file.write("          + only_suv  * ALL_SUV\n")                                     
                    if vt=='pu': 
                        file.write("          + van_pu    * OWN_VAN\n")
                        file.write("          + suv_pu    * OWN_SUV\n")
                        file.write("          + pu_pu     * OWN_PU\n")
                        file.write("          + pu_mc     * OWN_MC\n") 
                        file.write("          + only_pu   * ALL_PU\n")                    
                    if vt=='mc': 
                        file.write("          + van_mc    * OWN_VAN\n")                
                        file.write("          + suv_mc    * OWN_SUV\n")                
                        file.write("          + pu_mc     * OWN_PU\n")                
                        file.write("          + mc_mc     * OWN_MC\n")   
                        file.write("          + only_mc   * ALL_MC\n")             

                    # household interactions by fuel type                
                    if ft2=='hyb':
                        file.write("          + hyb_hyb   * OWN_HYB\n")
                        file.write("          + hyb_ev    * OWN_EV\n")                    
                    if ft2=='ev':
                        file.write("          + hyb_ev    * OWN_HYB\n")    
                                                      
                    if ft1=='bev': 
                        file.write("          + only_bev  * ALL_BEV\n")         

                    # electric specific measures
                    if ft2=='ev':
                        file.write("          + ln_chpc_ev   * LN_CHG_PC\n")
                        
                    if ft1=='bev':
                        file.write("          + crange     * " + str(evRange[i]) + "\n")
                        file.write("          + crangeltwk   * (iflt(" + str(evRange[i]) + ",2*DSTTOWKAVG)) \n")
                        
                    # alternative specific attributes
                    file.write("          + ln_nmods   * " + str(np.log(1+numModels[i])) + "\n")
                    file.write("          + ln_makes   * " + str(np.log(1+numMakes[i])) + "\n")
                    file.write("          + cmpg       * " + str(mpg[i]) + "\n")
                    file.write("          + ccharge240 * " + str(charge240[i]) + "\n")
                    
                    file.write("          + cprice0    * (INC025 * " + str(newPrice[i]) + ")\n")
                    file.write("          + cprice25   * (INC2550 * " + str(newPrice[i]) + ")\n")
                    file.write("          + cprice50   * (INC50100 * " + str(newPrice[i]) + ")\n")
                    file.write("          + cprice100  * (INC100150 * " + str(newPrice[i]) + ")\n")
                    file.write("          + cprice150  * (INC150P * " + str(newPrice[i]) + ")\n")  
                    file.write("          + cpricem    * (INC_MISS * " + str(newPrice[i]) + ")\n")  
                    
                # new line
                file.write("\n")
                

In [10]:
# start from run 826, but introduce geographic constants
with open('vehtype902.util', 'w') as file:
    
    i = 0
    for v in range(1,6):
        for f in range(1,6):
            for a in range(1,21):
                i += 1
                altnum = str(i)
                alt= str(1000*v + 100*f + a)
                vt = vehtypes[v-1]
                vt2 = vehtypes2[v-1]
                ft1 = fueltypes1[f-1]
                ft2 = fueltypes2[f-1]
                ft3 = fueltypes3[f-1]
                age = str(a)
                
                # add a comment saying what the alternative is
                file.write("- " + vehtype_names[v-1] + '-' + fueltype_names[f-1] + '-Age ' + age + '\n')

                # define the utility equation
                file.write("UTIL(" + altnum + ") = 0 \n") 
                
                # only write the rest of the utility equation if the alternative is available
                if numModels[i]>0: 

                    # 1-dimensional ASCs for vehicle type, fuel type and age
                    if v>1: 
                        file.write("          + " + vt + "\n")
                    if f>1: 
                        file.write("          + " + ft1 + "\n")
                    if a>1: 
                        file.write("          + age" + age + "\n")

                    # car is the reference alternative
                    if vt!='car':
                        file.write("          + vhgtdr_" + vt + " * VHGTDR \n")
                        file.write("          + nchld_" + vt + "  * NUM_CHLD3 \n")
                        file.write("          + den3_" + vt + "   * DEN3 \n")
                        file.write("          + den4_" + vt + "   * DEN4 \n")
                        file.write("          + den5_" + vt + "   * DEN5 \n")
                        file.write("          + den6_" + vt + "   * DEN6 \n")
                        file.write("          + den7_" + vt + "   * DEN7 \n")
                        file.write("          + den8_" + vt + "   * DEN8 \n")
                        
                        file.write("          + sfo_"  + vt + "   * SFO \n")
                        file.write("          + san_"  + vt + "   * SAN \n")
                        file.write("          + atl_"  + vt + "   * ATL \n")
                        file.write("          + smsa_" + vt + "   * SMSA \n")
                        file.write("          + rural_" + vt + "  * RURAL \n")

                    # gas (and diesel) is the reference alternative
                    if ft3!='gas':
                        file.write("          + vhgtdr_" + ft3 + " * VHGTDR \n")
                        file.write("          + nchld_" + ft3 + "  * NUM_CHLD3 \n")
                        
                        # group some of the EV density terms for noisy data
                        if ft3=='ev':
                            file.write("          + den34_" + ft3 + "  * DEN34 \n")
                            file.write("          + den5_" + ft3 + "   * DEN5 \n")
                            file.write("          + den6_" + ft3 + "   * DEN6 \n")
                            file.write("          + den78_" + ft3 + "  * DEN78 \n")
                        else:
                            file.write("          + den3_" + ft3 + "   * DEN3 \n")
                            file.write("          + den4_" + ft3 + "   * DEN4 \n")
                            file.write("          + den5_" + ft3 + "   * DEN5 \n")
                            file.write("          + den6_" + ft3 + "   * DEN6 \n")
                            file.write("          + den7_" + ft3 + "   * DEN7 \n")
                            file.write("          + den8_" + ft3 + "   * DEN8 \n")
                                                       
                        
                        file.write("          + sfo_"  + ft3 + "   * SFO \n")
                        file.write("          + san_"  + ft3 + "   * SAN \n")
                        file.write("          + atl_"  + ft3 + "   * ATL \n")
                        file.write("          + smsa_" + ft3 + "   * SMSA \n")
                        file.write("          + rural_" + ft3 + "  * RURAL \n")

                    # all ages are included
                    file.write("          + i025_age   * INC025a(" + age + ")\n")
                    file.write("          + i2550_age  * INC2550a(" + age + ")\n")
                    file.write("          + i100_age   * INC100150a(" + age + ")\n")
                    file.write("          + i150p_age  * INC150Pa(" + age + ")\n")
                    file.write("          + imiss_age  * INC_MISSa(" + age + ")\n")   
                    file.write("          + oneveh_age * ONEVEHa(" + age + ")\n")
                    file.write("          + vhgtdr_age * VHGTDRa(" + age + ")\n")
                    file.write("          + nchld_age  * NUM_CHLD3a(" + age + ")\n")
                    file.write("          + dstwkt_age * DSTWKTa(" + age + ")\n")
                    
                    file.write("          + sfo_age    * SFOa(" + age + ")\n")
                    file.write("          + san_age    * SANa(" + age + ")\n")
                    file.write("          + atl_age    * ATLa(" + age + ")\n")
                    file.write("          + smsa_age   * SMSAa(" + age + ")\n")
                    file.write("          + rural_age  * RURALa(" + age + ")\n")    

                    # household interactions by vehicle type                
                    if vt=='van':
                        file.write("          + van_van   * OWN_VAN\n")
                        file.write("          + van_suv   * OWN_SUV\n")
                        file.write("          + van_pu    * OWN_PU\n")
                        file.write("          + van_mc    * OWN_MC\n")  
                        file.write("          + only_van  * ALL_VAN\n")                         
                    if vt=='suv':
                        file.write("          + van_suv   * OWN_VAN\n")
                        file.write("          + suv_suv   * OWN_SUV\n")
                        file.write("          + suv_pu    * OWN_PU\n")
                        file.write("          + suv_mc    * OWN_MC\n")      
                        file.write("          + only_suv  * ALL_SUV\n")                                     
                    if vt=='pu': 
                        file.write("          + van_pu    * OWN_VAN\n")
                        file.write("          + suv_pu    * OWN_SUV\n")
                        file.write("          + pu_pu     * OWN_PU\n")
                        file.write("          + pu_mc     * OWN_MC\n") 
                        file.write("          + only_pu   * ALL_PU\n")                    
                    if vt=='mc': 
                        file.write("          + van_mc    * OWN_VAN\n")                
                        file.write("          + suv_mc    * OWN_SUV\n")                
                        file.write("          + pu_mc     * OWN_PU\n")                
                        file.write("          + mc_mc     * OWN_MC\n")   
                        file.write("          + only_mc   * ALL_MC\n")             

                    # household interactions by fuel type                
                    if ft2=='hyb':
                        file.write("          + hyb_hyb   * OWN_HYB\n")
                        file.write("          + hyb_ev    * OWN_EV\n")                    
                    if ft2=='ev':
                        file.write("          + hyb_ev    * OWN_HYB\n")    
                                                      
                    if ft1=='bev': 
                        file.write("          + only_bev  * ALL_BEV\n")         

                    # electric specific measures
                    if ft2=='ev':
                        file.write("          + ln_chpc_ev   * LN_CHG_PC\n")
                        
                    if ft1=='bev':
                        file.write("          + crange     * " + str(evRange[i]) + "\n")
                        file.write("          + crangeltwk   * (iflt(" + str(evRange[i]) + ",2*DSTTOWKAVG)) \n")
                        
                    # alternative specific attributes
                    file.write("          + ln_nmods   * " + str(np.log(1+numModels[i])) + "\n")
                    file.write("          + ln_makes   * " + str(np.log(1+numMakes[i])) + "\n")
                    file.write("          + cmpg       * " + str(mpg[i]) + "\n")
                    file.write("          + ccharge240 * " + str(charge240[i]) + "\n")
                    
                    file.write("          + cprice0    * (INC025 * " + str(newPrice[i]) + ")\n")
                    file.write("          + cprice25   * (INC2550 * " + str(newPrice[i]) + ")\n")
                    file.write("          + cprice50   * (INC50100 * " + str(newPrice[i]) + ")\n")
                    file.write("          + cprice100  * (INC100150 * " + str(newPrice[i]) + ")\n")
                    file.write("          + cprice150  * (INC150P * " + str(newPrice[i]) + ")\n")  
                    file.write("          + cpricem    * (INC_MISS * " + str(newPrice[i]) + ")\n")  
                    
                # new line
                file.write("\n")
                

In [6]:
# drop time to charge and clean up fuel type
with open('vehtype910.util', 'w') as file:
    
    i = 0
    for v in range(1,6):
        for f in range(1,6):
            for a in range(1,21):
                i += 1
                altnum = str(i)
                alt= str(1000*v + 100*f + a)
                vt = vehtypes[v-1]
                vt2 = vehtypes2[v-1]
                ft1 = fueltypes1[f-1]
                ft2 = fueltypes2[f-1]
                ft3 = fueltypes3[f-1]
                age = str(a)
                
                # add a comment saying what the alternative is
                file.write("- " + vehtype_names[v-1] + '-' + fueltype_names[f-1] + '-Age ' + age + '\n')

                # define the utility equation
                file.write("UTIL(" + altnum + ") = 0 \n") 
                
                # only write the rest of the utility equation if the alternative is available
                if numModels[i]>0: 

                    # 1-dimensional ASCs for vehicle type, fuel type and age
                    if v>1: 
                        file.write("          + " + vt + "\n")
                    if f>1: 
                        file.write("          + " + ft1 + "\n")
                    if a>1: 
                        file.write("          + age" + age + "\n")

                    # car is the reference alternative
                    if vt!='car':
                        file.write("          + vhgtdr_" + vt + " * VHGTDR \n")
                        file.write("          + nchld_" + vt + "  * NUM_CHLD3 \n")
                        file.write("          + den3_" + vt + "   * DEN3 \n")
                        file.write("          + den4_" + vt + "   * DEN4 \n")
                        file.write("          + den5_" + vt + "   * DEN5 \n")
                        file.write("          + den6_" + vt + "   * DEN6 \n")
                        file.write("          + den7_" + vt + "   * DEN7 \n")
                        file.write("          + den8_" + vt + "   * DEN8 \n")
                        
                        file.write("          + sfo_"  + vt + "   * SFO \n")
                        file.write("          + san_"  + vt + "   * SAN \n")
                        file.write("          + atl_"  + vt + "   * ATL \n")
                        file.write("          + sea_"  + vt + "   * SEA \n")
                        file.write("          + det_"  + vt + "   * DET \n")
                        file.write("          + msp_"  + vt + "   * MSP \n")
                        file.write("          + dca_"  + vt + "   * DCA \n")
                        file.write("          + oreg_" + vt + "   * OREG \n")
                        file.write("          + ohio_" + vt + "   * OHIO \n")
                        file.write("          + smsa_" + vt + "   * SMSA \n")
                        file.write("          + rural_" + vt + "  * RURAL \n")

                    # gas (and diesel) is the reference alternative
                    if ft3!='gas':
                        file.write("          + vhgtdr_" + ft3 + " * VHGTDR \n")
                        file.write("          + nchld_" + ft3 + "  * NUM_CHLD3 \n")
                        # group some of the EV density terms for noisy data
                        if ft3=='ev':
                            file.write("          + den34_" + ft3 + "  * DEN34 \n")
                            file.write("          + den5_" + ft3 + "   * DEN5 \n")
                            file.write("          + den6_" + ft3 + "   * DEN6 \n")
                            file.write("          + den78_" + ft3 + "  * DEN78 \n")
                        else:
                            file.write("          + den3_" + ft3 + "   * DEN3 \n")
                            file.write("          + den4_" + ft3 + "   * DEN4 \n")
                            file.write("          + den5_" + ft3 + "   * DEN5 \n")
                            file.write("          + den6_" + ft3 + "   * DEN6 \n")
                            file.write("          + den7_" + ft3 + "   * DEN7 \n")
                            file.write("          + den8_" + ft3 + "   * DEN8 \n")
                        
                        file.write("          + sfo_"  + ft3 + "   * SFO \n")
                        file.write("          + san_"  + ft3 + "   * SAN \n")
                        file.write("          + atl_"  + ft3 + "   * ATL \n")
                        file.write("          + sea_"  + ft3 + "   * SEA \n")
                        file.write("          + det_"  + ft3 + "   * DET \n")
                        file.write("          + msp_"  + ft3 + "   * MSP \n")
                        file.write("          + dca_"  + ft3 + "   * DCA \n")
                        file.write("          + oreg_" + ft3 + "   * OREG \n")
                        file.write("          + ohio_" + ft3 + "   * OHIO \n")
                        file.write("          + smsa_" + ft3 + "   * SMSA \n")
                        file.write("          + rural_" + ft3 + "  * RURAL \n")

                    # all ages are included
                    file.write("          + i025_age   * INC025a(" + age + ")\n")
                    file.write("          + i2550_age  * INC2550a(" + age + ")\n")
                    file.write("          + i100_age   * INC100150a(" + age + ")\n")
                    file.write("          + i150p_age  * INC150Pa(" + age + ")\n")
                    file.write("          + imiss_age  * INC_MISSa(" + age + ")\n")   
                    file.write("          + oneveh_age * ONEVEHa(" + age + ")\n")
                    file.write("          + vhgtdr_age * VHGTDRa(" + age + ")\n")
                    file.write("          + nchld_age  * NUM_CHLD3a(" + age + ")\n")
                    file.write("          + dstwkt_age * DSTWKTa(" + age + ")\n")
                    
                    file.write("          + sfo_age    * SFOa(" + age + ")\n")
                    file.write("          + san_age    * SANa(" + age + ")\n")
                    file.write("          + atl_age    * ATLa(" + age + ")\n")
                    file.write("          + sea_age    * SEAa(" + age + ")\n")
                    file.write("          + det_age    * DETa(" + age + ")\n")
                    file.write("          + msp_age    * MSPa(" + age + ")\n")
                    file.write("          + dca_age    * DCAa(" + age + ")\n")
                    file.write("          + oreg_age   * OREGa(" + age + ")\n")
                    file.write("          + ohio_age   * OHIOa(" + age + ")\n")
                    file.write("          + smsa_age   * SMSAa(" + age + ")\n")
                    file.write("          + rural_age  * RURALa(" + age + ")\n")    

                    # household interactions by vehicle type                
                    if vt=='van':
                        file.write("          + van_van   * OWN_VAN\n")
                        file.write("          + van_suv   * OWN_SUV\n")
                        file.write("          + van_pu    * OWN_PU\n")
                        file.write("          + van_mc    * OWN_MC\n")  
                        file.write("          + only_van  * ALL_VAN\n")                         
                    if vt=='suv':
                        file.write("          + van_suv   * OWN_VAN\n")
                        file.write("          + suv_suv   * OWN_SUV\n")
                        file.write("          + suv_pu    * OWN_PU\n")
                        file.write("          + suv_mc    * OWN_MC\n")      
                        file.write("          + only_suv  * ALL_SUV\n")                                     
                    if vt=='pu': 
                        file.write("          + van_pu    * OWN_VAN\n")
                        file.write("          + suv_pu    * OWN_SUV\n")
                        file.write("          + pu_pu     * OWN_PU\n")
                        file.write("          + pu_mc     * OWN_MC\n") 
                        file.write("          + only_pu   * ALL_PU\n")                    
                    if vt=='mc': 
                        file.write("          + van_mc    * OWN_VAN\n")                
                        file.write("          + suv_mc    * OWN_SUV\n")                
                        file.write("          + pu_mc     * OWN_PU\n")                
                        file.write("          + mc_mc     * OWN_MC\n")   
                        file.write("          + only_mc   * ALL_MC\n")             

                    # household interactions by fuel type                
                    if ft2=='hyb':
                        file.write("          + hyb_hyb   * OWN_HYB\n")
                        file.write("          + hyb_ev    * OWN_EV\n")                    
                    if ft2=='ev':
                        file.write("          + hyb_ev    * OWN_HYB\n")    
                                                      
                    if ft1=='bev': 
                        file.write("          + only_bev  * ALL_BEV\n")         

                    # electric specific measures
                    if ft2=='ev':
                        file.write("          + ln_chpc_ev   * LN_CHG_PC\n")
                        
                    if ft1=='bev':
                        file.write("          + crange     * " + str(evRange[i]) + "\n")
                        file.write("          + crangeltwk   * (iflt(" + str(evRange[i]) + ",2*DSTTOWKAVG)) \n")
                        
                    # alternative specific attributes
                    file.write("          + ln_nmods   * " + str(np.log(1+numModels[i])) + "\n")
                    file.write("          + ln_makes   * " + str(np.log(1+numMakes[i])) + "\n")
                    file.write("          + cmpg       * " + str(mpg[i]) + "\n")
                    
                    file.write("          + cprice0    * (INC025 * " + str(newPrice[i]) + ")\n")
                    file.write("          + cprice25   * (INC2550 * " + str(newPrice[i]) + ")\n")
                    file.write("          + cprice50   * (INC50100 * " + str(newPrice[i]) + ")\n")
                    file.write("          + cprice100  * (INC100150 * " + str(newPrice[i]) + ")\n")
                    file.write("          + cprice150  * (INC150P * " + str(newPrice[i]) + ")\n")  
                    file.write("          + cpricem    * (INC_MISS * " + str(newPrice[i]) + ")\n")  
                    
                # new line
                file.write("\n")
                